In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

with open('Sarcasm_Headlines_Dataset.json') as read_file:
    jsondata=read_file.readlines()

headlines = []
Class = []
for i in jsondata:
    x=json.loads(i)
    headlines.append(x['headline'])
    Class.append(x['is_sarcastic'])

In [ ]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string 
import re

Tokens = []
Dictionary1 = []
Dictionary2 = []

for y in headlines:    
    y = re.sub(r'(\s)#\w+', r'\1', y)
    y = re.sub('[^a-zA-Z]', ' ', y)
    y = y.lower().split()
    y[:] = [x for x in y if x not in stopwords.words('english')]
    y = [ word for word in y if len(word) > 1 ]
    ps = PorterStemmer()
    y = [ps.stem(word) for word in y]
    Tokens = list(set(Tokens) | set(y))
    Dictionary2.append(y)
    y = " ".join([x for x in y])
    Dictionary1.append(y)

Df ={}
from collections import Counter
for i in Dictionary2:
    Df = Counter(Df) + Counter(i)
    
Df = Counter(dict(filter(lambda x: x[1] >= 10, Df.items())))
val = [x for x in Df.keys()]

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

pipe = Pipeline([('count', CountVectorizer(vocabulary=val)),
                 ('tfid', TfidfTransformer())]).fit(Dictionary1)

Tf_count = pipe['count'].transform(Dictionary1).toarray()
Tf_idf = pipe['tfid'].idf_
features = pipe.transform(Dictionary1).toarray()

import pandas as pd
Dataset = pd.DataFrame(features , columns = val)
Dataset['Class'] = Class

Dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
Training_data = Dataset [:int(len(Dataset)*0.8)]
Testing_data = Dataset [int(len(Dataset)*0.8):]

import math 
Attributes = Training_data.columns[:-1]

Mean_Variance = {}
for i in Training_data['Class'].unique():                       
    Class_set = Training_data[(Training_data['Class'] == i )]       
    mean_var = {}
    for j in Attributes:
        mean_var[j] = []
        mean_var[j].append(Class_set[j].mean())
        mean_var[j].append(math.pow(Class_set[j].std(), 2))
    Mean_Variance[i] = mean_var
    
def Probability(x , mean , variance):
    exponent = math.exp(-(math.pow(x - mean, 2) / (2 * variance)))
    return (exponent / (math.sqrt(2 * math.pi * variance)))

print(Mean_Variance)

{0: {'former': [0.0020997993209006262, 0.0008084347934317923], 'store': [0.0006091881495714701, 0.0002682090146636539], 'clerk': [0.0001695719470077448, 6.914135614890667e-05], 'sue': [0.0007620111077007105, 0.00033538363832839115], 'secret': [0.002215729335757652, 0.0009750716302353788], 'black': [0.004910350788183854, 0.0018327322934576947], 'code': [0.00040331419932890066, 0.00022756423632461627], 'minor': [0.0005409766480901912, 0.00023865062191374864], 'reviv': [0.0005365716338637152, 0.00024760510991266455], 'catch': [0.000741488390570543, 0.0003556334053058889], 'polit': [0.002816346965946491, 0.0012329077403474156], 'mood': [0.00018863538341767558, 0.00011006064842963885], 'better': [0.0020924138598268694, 0.0009499619279572139], 'wors': [0.000767050934538177, 0.0003695780445329392], 'mom': [0.003790547836789928, 0.0014878710278284982], 'start': [0.0016449016206374406, 0.0006566959673847221], 'fear': [0.0014589061310592173, 0.0006752646930062555], 'son': [0.0018504198973103188,

In [5]:
def preprocess(y):
    y = re.sub(r'(\s)#\w+', r'\1', y)
    y = re.sub('[^a-zA-Z]', ' ', y)
    y = y.lower().split()
    y[:] = [x for x in y if x not in stopwords.words('english')]
    ps = PorterStemmer()
    y = [ps.stem(word) for word in y]
    return y


In [6]:
def predict ():
    
    headline = entry1.get() 
    Test_features = preprocess(headline)
    
    Test_data = {}

    for i in val:
        if i in Test_features:
            Test_data[i] = 1
        else:
            Test_data[i] = 0
            
    P_0 = len(Training_data[(Training_data['Class'] == 0 )]) / len(Training_data.index)
    P_1 = len(Training_data[(Training_data['Class'] == 1 )]) / len(Training_data.index)
    for feature in Attributes:
        x = Test_data[feature]   
        if x != 0.0:
            y0 = Mean_Variance[0][feature]          
            y1 = Mean_Variance[1][feature]          
            P_0 *= Probability(x , y0[0] , y0[1])    
            P_1 *= Probability(x, y1[0] , y1[1])
    
    if (P_0 > P_1): 
        entry2.insert(END,"Not Sarcastic")
    else:
        entry2.insert(END,"Sarcastic")


In [7]:
def RefreshGUI():
    
    entry1.delete(0,'end')
    entry2.delete(1.0,END)

# This function is used to close the GUI window

def CloseGUI():
    
    win.destroy()


In [8]:
from tkinter import *                               # Built in library 

win = Tk()                                          # creates the GUI window
win.configure(background='SteelBlue4')              # sets the window colour
win.title("Sarcasm Detector ") # used to set the title of the window
win.geometry("550x400")                            # used to set the size of the window 

greeting = Label(win,text="SARCASM DETECTOR",bg='SteelBlue4',fg="black",height=2,font=("Arial",16,"bold")) 
greeting.place(x=145,y=10)                         # used to set the position

label1 = Label(win,text="Type a sentence",bg='SteelBlue4',fg='black', font=('helvetica', 12, 'bold')) # used to create the input statement
entry1 = Entry(win)                                # used to create the input box 

label1.place(x=200,y=90)                           # used to set the position of the input statement
entry1.place(x=110,y=120,width=310,height=23)      # used to set the position of the input box

button = Button(win,text='Find Result', command=predict ,bg='light grey', fg='black', font=('helvetica', 9, 'bold')) 
button.place(x=220,y=190,width=100)            # used to set the position of the button 
 
entry2 = Text(win, height=2)                       # creates the text widget that stores retrieved documents
entry2.place(x=170,y=250,width=190)                # used to set the position of the widget
entry2.config(state=NORMAL)

button1 = Button(win,text='Clear',width=10, command=RefreshGUI ,bg='light grey', fg='black', font=('helvetica', 9, 'bold'))
button1.place(x=180,y=350)                        # places the REFRESH button

button2 = Button(win,text='Exit',width=10, command=CloseGUI , bg='light grey', fg='black', font=('helvetica', 9, 'bold'))
button2.place(x=280,y=350)                        # places the EXIT button

win.mainloop()